In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
from langchain_anthropic import ChatAnthropic

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_vertexai import ChatVertexAI
model = ChatVertexAI(model='gemini-1.5-pro')

# os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
# model = ChatAnthropic(model='claude-3-5-sonnet-20240620')
# openai_api_key = os.getenv('OPENAI_API_KEY')
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./my_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./my_resume.md')

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=model
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=model
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=model
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=model
)

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [14]:
# Create crew
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [15]:
# Edit params here
job_application_inputs = {
    'job_posting_url': 'https://www.tesla.com/careers/search/job/221939?source=LinkedIn',
    'github_url': 'https://github.com/Yashvishe13',
    'personal_writeup': """Yash is a Data Analyst with one year of experience in Python, SQL, Excel, Superset, Apache, AWS, and Tableau at Media.net, with a passion for extracting insights from data.He has developed several freelancing projects involving technologies like LLMs, GCP, AWS, and APIs. Seeking to leverage his technical and problem-solving skills in a data science or generative AI role while learning from industry experts."""
}

In [16]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [2024-07-05 17:26:41][DEBUG]: == Working Agent: Tech Job Researcher
 [2024-07-05 17:26:41][INFO]: == Starting Task: Analyze the job posting URL provided (https://www.tesla.com/careers/search/job/221939?source=LinkedIn) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [2024-07-05 17:26:41][DEBUG]: == [Tech Job Researcher] Task output: 


 [2024-07-05 17:26:41][DEBUG]: == Working Agent: Personal Profiler for Engineers
 [2024-07-05 17:26:41][INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/Yashvishe13) URLs, and personal write-up (Yash is a Data Analyst with one year of experience in Python, SQL, Excel, Superset, Apache, AWS, and Tableau at Media.net, with a passion for extracting insights from data.He has developed several freelancing projects involving technologies like LLMs, GCP, AWS, and APIs. Seeking to leverage his technica

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

YASH VISHE
website.io LinkedIn yvishe@ucsd.edu +91 81046 52268

### EDUCATION

University of California San Diego La Jolla, CA
Master of Science in Data Science Sep. 2024 - Jun. 2026

University of Mumbai Mumbai, India
Bachelor of Engineering in Computer Engineering Aug. 2019 - Jun. 2023

### TECHNICAL SKILLS

**Languages/Tools:** Python, C++, SQL, Tableau, Git, GitLab, AWS, Airflow, Superset, GCP

**Libraries/Frameworks:**  TensorFlow, Keras, Scikit-learn, Pandas, Matplotlib, XGBoost, Flask, SpaCy, NLTK, LangChain, LangGraph, KnowledgeGraphs, ChromaDB

**Skills:**

* **Large-Scale Model Training & AI System Design:** Expertise in designing, implementing, and evaluating large-scale machine learning models and AI systems, leveraging frameworks like TensorFlow and Keras. Proven ability to apply cutting-edge AI techniques and research to real-world problems. 
* **Data Engineering & Analysis:** Proficient in data engineering techniques, including ETL pipeline development using tools like Airflow and AWS Redshift. Skilled in data analysis, visualization (Tableau, Matplotlib), and cleaning, enabling data-driven insights.
* **Natural Language Processing (NLP):** Experienced in NLP techniques, including Named Entity Recognition, text summarization, and document query agent development, utilizing libraries like SpaCy, NLTK, and LangChain. 
* **Communication & Collaboration:**  Effective communicator with strong collaboration skills, demonstrated through successful project leadership and teamwork experience.

### PROFESSIONAL EXPERIENCE

**Media.net Mumbai, India Data Analyst (Python, SQL, AWS Redshift, Airflow, Superset, Tableau) July. 2023 - Aug. 2024**

• Designed and implemented efficient ETL pipelines, migrating 46 alerts from Apache Superset to Apache Airflow to reduce the load on AWS Redshift, resulting in a 38% improvement in data retrieval and analysis speed for both ProdOps and DevOps teams.
• Engineered data solutions by creating and managing various buckets on AWS S3 to facilitate storage and access of CSV files and images for alerts, significantly enhancing data accessibility and storage efficiency.
• Collaborated with senior leadership to analyze one year of data on Tableau, identifying key sources for traffic volume growth and providing actionable insights leading to data-driven decision-making.
• Developed a dynamic bid adjustment algorithm in Python, strategically increasing bids with volume growth and reducing them otherwise, achieving a 13% increase in overall traffic volume, showcasing expertise in data-driven optimization.

**Neumann Fornaxx New Delhi, India Software Development Intern (Python, SQL, HTML, CSS) Nov. 2022 - Feb. 2023**

• Led a team of three interns, demonstrating leadership and teamwork, to successfully deliver a project at Neumann Fornax ahead of schedule.
• Gathered and processed over 100,000 rows of data from Zomato and Swiggy using Python CRONs and BeautifulSoup for data analysis, showcasing proficiency in web scraping and data extraction.
• Designed and implemented an automated system to delete duplicate data, reducing database size by 62%, highlighting skills in data cleaning and optimization for efficient data management.

### PERSONAL PROJECTS

**Document Query Agent using Multi-Modal RAG — Langchain, Python, VertexAI, ChromaDB (GitHub)**
• Designed and implemented a document query agent, showcasing expertise in AI system design using Vertex AI, ChromaDB, and LangChain. 
• Employed multi-modal self-reflective RAG for precise data retrieval and integrated Tavily web search agent to mitigate hallucination issues, demonstrating an understanding of cutting-edge AI techniques.
• Programmed the Gemini-1.5-pro model to generate summaries of unstructured data, highlighting experience in large-scale model training and data enhancement for improved AI system performance.

**Community Analysis using Graph Networks — Python, SpaCy, NetworkX (Github)**
• Developed a system to parse large text data, extracting and analyzing relationships using Named Entity Recognition, showcasing experience with data processing and analysis techniques relevant to large-scale data sets.
• Created interactive network graphs and employed community detection algorithms, demonstrating expertise in data visualization and complex network analysis, valuable skills for understanding and improving AI systems.

**Book Recommendation Engine — Python, NLTK, Sklearn (Github)**
• Designed and implemented a book recommendation system using machine learning algorithms like cosine similarity, TFIdvectorizer, and collaborative filtering.
• Scraped and processed data from GoodReads, showcasing data collection and processing skills relevant to training and improving AI models.
• This project highlights experience in building and optimizing machine learning models, a crucial aspect of large-scale model training and AI system design. 

### RESEARCH AND PUBLICATIONS

**Enhancing Brain Tumor Detection Using Multimodal Approach** | May 2024
*IEEE · International Conference on Artificial Intelligence and Machine Learning Applications*
This research focused on optimizing deep learning models for large-scale medical image analysis, specifically brain tumor detection. I trained and evaluated various architectures, including ResNet50, InceptionNet, and DenseNet, employing techniques like ensemble learning (weighted majority averaging) to achieve high accuracy (96.33%).  Furthermore, I investigated the impact of loss functions, comparing Focal Tversky Loss and Dice Loss, to minimize false negatives.  The project also involved developing a segmentation model (ResUNet) for precise tumor localization within MRI scans.